In [1]:
%reset -fs
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

In [3]:
bet_df = pd.read_csv('betting_data.csv', index_col=0)
bet_df

,Date,Name,Team,Position,Opp,Home/Away,Line,Moneyline,Projection,Diff,Pick,Result,pick_over_under,seven_day_accuracy,successful_pick
0,2021-10-21,Bogdan,ATL,G,DAL,vs.,15.5,-115,16.57,1.07,16.0,11,o,0.47,0
1,2021-10-23,Bogdan,ATL,G,CLE,@,14.5,-115,15.29,0.79,14.5,5,o,0.45,0
2,2021-10-25,Bogdan,ATL,G,DET,vs.,15.5,-115,12.85,-2.65,13.5,14,u,0.45,0
3,2021-10-28,Bogdan,ATL,G,WAS,@,12.5,-105,16.72,4.22,12.5,14,o,0.42,1
4,2021-10-30,Bogdan,ATL,G,PHI,@,12.5,-113,15.71,3.21,12.5,13,o,0.45,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4621,2021-12-04,Zeke Nnaji,DEN,F,NYK,@,5.5,-105,2.91,-2.59,5.5,21,u,0.55,0
4622,2021-12-06,Zeke Nnaji,DEN,F,CHI,@,7.5,115,3.35,-4.15,7.5,3,u,0.51,1
4623,2021-10-28,Ziaire Williams,MEM,G,GSW,@,5.5,-130,4.33,-1.17,5.5,4,u,0.42,1
4624,2022-01-17,Ziaire Williams,MEM,G,CHI,vs.,8.5,-105,6.11,-2.39,8.5,2,u,0.58,1


In [4]:
stat_df = pd.read_csv('stathead_data.csv', index_col=0)

In [5]:
stat_df

,Player,Age,Pos,Tm,Unnamed: 5,Opp,Unnamed: 7,Date,GS,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,TS%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,BPM
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,LaMarcus Aldridge\aldrila01,36-092,F-C,BRK,@,MIL,L,2021-10-19,0,16,0,2,0.000,0,2,0.000,0,0,NaN,1,4,0.250,0.133,0,3,3,1,0,1,1,2,1,-1.1,-15.2
2,Grayson Allen\allengr01,26-011,G,MIL,NaN,BRK,W,2021-10-19,1,28,3,10,0.300,0,0,NaN,3,10,0.300,1,1,1.000,0.479,0,4,4,6,2,0,0,0,10,11.6,6.4
3,Giannis Antetokounmpo\antetgi01,26-317,F-G,MIL,NaN,BRK,W,2021-10-19,1,31,12,25,0.480,11,21,0.524,1,4,0.250,7,9,0.778,0.552,5,9,14,7,1,2,4,4,32,26.4,11.4
4,Thanasis Antetokounmpo\antetth01,29-093,F,MIL,NaN,BRK,W,2021-10-19,0,12,2,3,0.667,2,3,0.667,0,0,NaN,2,2,1.000,0.773,1,3,4,1,0,1,0,1,6,7.3,8.0
5,Carmelo Anthony\anthoca01,37-143,F,LAL,NaN,GSW,L,2021-10-19,0,26,3,9,0.333,1,5,0.200,2,4,0.500,1,2,0.500,0.455,0,4,4,2,0,0,1,2,9,4.3,-3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14998,Ziaire Williams\willizi02,20-133,F,MEM,@,DAL,L,2022-01-23,1,23,4,13,0.308,3,5,0.600,1,8,0.125,0,0,NaN,0.346,1,0,1,1,0,0,0,5,9,0.9,-9.6
14999,Delon Wright\wrighde01,29-272,G,ATL,@,CHO,W,2022-01-23,0,20,2,6,0.333,1,2,0.500,1,4,0.250,1,2,0.500,0.436,0,4,4,4,4,0,2,3,6,7.0,4.4
15000,McKinley Wright IV\wrighmc01,23-090,G,MIN,NaN,BRK,W,2022-01-23,0,1,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,NaN,0,0,0,1,0,0,0,0,0,0.7,14.1


In [8]:
stat_df.rename(columns={'Unnamed: 5': 'Home/Away', 'Unnamed: 7':'Game Result'}, inplace=True)

In [9]:
stat_df

,Player,Age,Pos,Tm,Home/Away,Opp,Game Result,Date,GS,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,TS%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,BPM
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,LaMarcus Aldridge\aldrila01,36-092,F-C,BRK,@,MIL,L,2021-10-19,0,16,0,2,0.000,0,2,0.000,0,0,NaN,1,4,0.250,0.133,0,3,3,1,0,1,1,2,1,-1.1,-15.2
2,Grayson Allen\allengr01,26-011,G,MIL,NaN,BRK,W,2021-10-19,1,28,3,10,0.300,0,0,NaN,3,10,0.300,1,1,1.000,0.479,0,4,4,6,2,0,0,0,10,11.6,6.4
3,Giannis Antetokounmpo\antetgi01,26-317,F-G,MIL,NaN,BRK,W,2021-10-19,1,31,12,25,0.480,11,21,0.524,1,4,0.250,7,9,0.778,0.552,5,9,14,7,1,2,4,4,32,26.4,11.4
4,Thanasis Antetokounmpo\antetth01,29-093,F,MIL,NaN,BRK,W,2021-10-19,0,12,2,3,0.667,2,3,0.667,0,0,NaN,2,2,1.000,0.773,1,3,4,1,0,1,0,1,6,7.3,8.0
5,Carmelo Anthony\anthoca01,37-143,F,LAL,NaN,GSW,L,2021-10-19,0,26,3,9,0.333,1,5,0.200,2,4,0.500,1,2,0.500,0.455,0,4,4,2,0,0,1,2,9,4.3,-3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14998,Ziaire Williams\willizi02,20-133,F,MEM,@,DAL,L,2022-01-23,1,23,4,13,0.308,3,5,0.600,1,8,0.125,0,0,NaN,0.346,1,0,1,1,0,0,0,5,9,0.9,-9.6
14999,Delon Wright\wrighde01,29-272,G,ATL,@,CHO,W,2022-01-23,0,20,2,6,0.333,1,2,0.500,1,4,0.250,1,2,0.500,0.436,0,4,4,4,4,0,2,3,6,7.0,4.4
15000,McKinley Wright IV\wrighmc01,23-090,G,MIN,NaN,BRK,W,2022-01-23,0,1,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,NaN,0,0,0,1,0,0,0,0,0,0.7,14.1


In [10]:
stat_df['Home/Away'] = stat_df['Home/Away'].fillna('Home')
stat_df['Home/Away'] = stat_df['Home/Away'].replace('@','Away')

In [11]:
stat_df

,Player,Age,Pos,Tm,Home/Away,Opp,Game Result,Date,GS,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,TS%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,GmSc,BPM
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,LaMarcus Aldridge\aldrila01,36-092,F-C,BRK,Away,MIL,L,2021-10-19,0,16,0,2,0.000,0,2,0.000,0,0,NaN,1,4,0.250,0.133,0,3,3,1,0,1,1,2,1,-1.1,-15.2
2,Grayson Allen\allengr01,26-011,G,MIL,Home,BRK,W,2021-10-19,1,28,3,10,0.300,0,0,NaN,3,10,0.300,1,1,1.000,0.479,0,4,4,6,2,0,0,0,10,11.6,6.4
3,Giannis Antetokounmpo\antetgi01,26-317,F-G,MIL,Home,BRK,W,2021-10-19,1,31,12,25,0.480,11,21,0.524,1,4,0.250,7,9,0.778,0.552,5,9,14,7,1,2,4,4,32,26.4,11.4
4,Thanasis Antetokounmpo\antetth01,29-093,F,MIL,Home,BRK,W,2021-10-19,0,12,2,3,0.667,2,3,0.667,0,0,NaN,2,2,1.000,0.773,1,3,4,1,0,1,0,1,6,7.3,8.0
5,Carmelo Anthony\anthoca01,37-143,F,LAL,Home,GSW,L,2021-10-19,0,26,3,9,0.333,1,5,0.200,2,4,0.500,1,2,0.500,0.455,0,4,4,2,0,0,1,2,9,4.3,-3.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14998,Ziaire Williams\willizi02,20-133,F,MEM,Away,DAL,L,2022-01-23,1,23,4,13,0.308,3,5,0.600,1,8,0.125,0,0,NaN,0.346,1,0,1,1,0,0,0,5,9,0.9,-9.6
14999,Delon Wright\wrighde01,29-272,G,ATL,Away,CHO,W,2022-01-23,0,20,2,6,0.333,1,2,0.500,1,4,0.250,1,2,0.500,0.436,0,4,4,4,4,0,2,3,6,7.0,4.4
15000,McKinley Wright IV\wrighmc01,23-090,G,MIN,Home,BRK,W,2022-01-23,0,1,0,0,NaN,0,0,NaN,0,0,NaN,0,0,NaN,NaN,0,0,0,1,0,0,0,0,0,0.7,14.1


In [13]:
team_stat_df = pd.read_csv('teamstats.csv', index_col=0)
team_stat_df

,Date,Tm,Unnamed: 3,Opp,Result,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,PTS,FG.1,FGA.1,FG%.1,2P.1,2PA.1,2P%.1,3P.1,3PA.1,3P%.1,FT.1,FTA.1,FT%.1,PTS.1
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2021-10-19,GSW,@,LAL,W 121-114,240,41,93,0.441,27,54,0.500,14,39,0.359,25,30,0.833,121,45,95,0.474,30,53,0.566,15,42,0.357,9,19,0.474,114
2,2021-10-19,LAL,NaN,GSW,L 114-121,240,45,95,0.474,30,53,0.566,15,42,0.357,9,19,0.474,114,41,93,0.441,27,54,0.500,14,39,0.359,25,30,0.833,121
3,2021-10-19,BRK,@,MIL,L 104-127,240,37,84,0.440,20,52,0.385,17,32,0.531,13,23,0.565,104,48,105,0.457,31,60,0.517,17,45,0.378,14,18,0.778,127
4,2021-10-19,MIL,NaN,BRK,W 127-104,240,48,105,0.457,31,60,0.517,17,45,0.378,14,18,0.778,127,37,84,0.440,20,52,0.385,17,32,0.531,13,23,0.565,104
5,2021-10-20,CHO,NaN,IND,W 123-122,240,46,107,0.430,33,76,0.434,13,31,0.419,18,27,0.667,123,42,90,0.467,25,43,0.581,17,47,0.362,21,24,0.875,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1396,2022-01-23,SAS,NaN,PHI,L 109-115,240,42,89,0.472,32,59,0.542,10,30,0.333,15,22,0.682,109,44,92,0.478,37,76,0.487,7,16,0.438,20,23,0.870,115
1397,2022-01-23,POR,@,TOR,W 114-105,240,45,85,0.529,27,53,0.509,18,32,0.563,6,13,0.462,114,39,98,0.398,21,44,0.477,18,54,0.333,9,14,0.643,105
1398,2022-01-23,TOR,NaN,POR,L 105-114,240,39,98,0.398,21,44,0.477,18,54,0.333,9,14,0.643,105,45,85,0.529,27,53,0.509,18,32,0.563,6,13,0.462,114


In [15]:
team_stat_df.rename(columns={'Unnamed: 3': 'Home/Away'}, inplace=True)
team_stat_df['Home/Away'] = team_stat_df['Home/Away'].fillna('Home')
team_stat_df['Home/Away'] = team_stat_df['Home/Away'].replace('@','Away')

In [16]:
team_stat_df

,Date,Tm,Home/Away,Opp,Result,MP,FG,FGA,FG%,2P,2PA,2P%,3P,3PA,3P%,FT,FTA,FT%,PTS,FG.1,FGA.1,FG%.1,2P.1,2PA.1,2P%.1,3P.1,3PA.1,3P%.1,FT.1,FTA.1,FT%.1,PTS.1
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2021-10-19,GSW,Away,LAL,W 121-114,240,41,93,0.441,27,54,0.500,14,39,0.359,25,30,0.833,121,45,95,0.474,30,53,0.566,15,42,0.357,9,19,0.474,114
2,2021-10-19,LAL,Home,GSW,L 114-121,240,45,95,0.474,30,53,0.566,15,42,0.357,9,19,0.474,114,41,93,0.441,27,54,0.500,14,39,0.359,25,30,0.833,121
3,2021-10-19,BRK,Away,MIL,L 104-127,240,37,84,0.440,20,52,0.385,17,32,0.531,13,23,0.565,104,48,105,0.457,31,60,0.517,17,45,0.378,14,18,0.778,127
4,2021-10-19,MIL,Home,BRK,W 127-104,240,48,105,0.457,31,60,0.517,17,45,0.378,14,18,0.778,127,37,84,0.440,20,52,0.385,17,32,0.531,13,23,0.565,104
5,2021-10-20,CHO,Home,IND,W 123-122,240,46,107,0.430,33,76,0.434,13,31,0.419,18,27,0.667,123,42,90,0.467,25,43,0.581,17,47,0.362,21,24,0.875,122
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1396,2022-01-23,SAS,Home,PHI,L 109-115,240,42,89,0.472,32,59,0.542,10,30,0.333,15,22,0.682,109,44,92,0.478,37,76,0.487,7,16,0.438,20,23,0.870,115
1397,2022-01-23,POR,Away,TOR,W 114-105,240,45,85,0.529,27,53,0.509,18,32,0.563,6,13,0.462,114,39,98,0.398,21,44,0.477,18,54,0.333,9,14,0.643,105
1398,2022-01-23,TOR,Home,POR,L 105-114,240,39,98,0.398,21,44,0.477,18,54,0.333,9,14,0.643,105,45,85,0.529,27,53,0.509,18,32,0.563,6,13,0.462,114
